In [1]:
import yfinance as yf

ticker = 'BTC-USD'
df = yf.download(ticker)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-09-04,57430.347656,58511.570312,55673.164062,57971.539062,57971.539062,35627680312
2024-09-05,57971.703125,58300.582031,55712.453125,56160.488281,56160.488281,31030280656
2024-09-06,56160.191406,56976.109375,52598.699219,53948.753906,53948.753906,49361693566


In [3]:
df = df.loc['2018-01-01':].copy()

In [5]:
df['change_tomorrow'] = df['Adj Close'].pct_change(-1)
df.change_tomorrow = df.change_tomorrow * -1
df.change_tomorrow = df.change_tomorrow * 100

In [7]:
df = df.dropna().copy()
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow
Date,,,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,13657.200195,10291200000,8.843216
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,14982.099609,16846600192,1.440039
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,15201.000000,16871900160,2.552696
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,15599.200195,21783199744,10.501161
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,17429.500000,23840899072,0.556285
...,...,...,...,...,...,...,...
2024-09-03,59106.191406,59815.058594,57425.167969,57431.023438,57431.023438,26666961053,0.932381
2024-09-04,57430.347656,58511.570312,55673.164062,57971.539062,57971.539062,35627680312,-3.224777
2024-09-05,57971.703125,58300.582031,55712.453125,56160.488281,56160.488281,31030280656,-4.099695


In [9]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

In [11]:
from sklearn.model_selection import TimeSeriesSplit

ts = TimeSeriesSplit(test_size=200)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model_dt = RandomForestRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]
    
    model_dt.fit(X_train, y_train)
    
    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)
    
    error_mse_list.append(error_mse)

In [15]:
error_mse_list


[16.195958333110138,
 10.28762976166315,
 6.355002003104143,
 5.977645776896398,
 11.220380148169347]

In [17]:
#Anchored Walk Forward evaluation in backtesting

In [21]:
from backtesting import Backtest, Strategy

In [23]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    n_train = 600
    coef_retrain = 200
    
    def init(self):
        self.model = RandomForestRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = self.data.df.iloc[:self.n_train, :-1]
        y_train = self.data.df.iloc[:self.n_train, -1]
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [25]:
class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()

In [29]:
from backtesting import Backtest
bt = Backtest(df, WalkForwardAnchored, cash=100000, commission=.002, exclusive_orders=True)

In [31]:
import multiprocessing as mp
mp.set_start_method('fork')

RuntimeError: context has already been set

In [33]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

Backtest.optimize:   0%|          | 0/500 [00:00<?, ?it/s]

C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [4, -6] before, using random point [4, -6]
  warnings.warn(
C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [3, -6] before, using random point [2, -5]
  warnings.warn(
C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [3, -6] before, using random point [2, -2]
  warnings.warn(
C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [3, -6] before, using random point [3, -5]
  warnings.warn(
C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [3, -6] before, using random point [0, -6]
  warnings.warn(
C:\Users\Dell\anaconda3\Lib\site-packages\sko

,limit_buy,limit_sell,Return [%]
9,2,-6,291.096237
15,3,-6,238.942538
0,0,-6,226.676380
14,2,-1,-100.000000
25,5,-5,-100.000000
24,4,-1,-100.000000
23,4,-2,-100.000000
22,4,-3,-100.000000
21,4,-5,-100.000000
20,4,-6,-100.000000


In [45]:
code = """
from backtesting import Strategy
from sklearn.tree import DecisionTreeRegressor

class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    n_train = 600
    coef_retrain = 200
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = self.data.df.iloc[:self.n_train, :-1]
        y_train = self.data.df.iloc[:self.n_train, -1]
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :-1]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass


class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()
            
            
class WalkForwardUnanchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < self.n_train:
            return
        
        # we retrain the model each 200 days
        if len(self.data) % self.coef_retrain == 0:
            X_train = self.data.df.iloc[-self.n_train:, :-1]
            y_train = self.data.df.iloc[-self.n_train:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()
"""
# Write the code to strategies.py
with open('strategies.py', 'w') as file:
    file.write(code)

In [47]:
#Unanchored Walk Forward

In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import strategies

In [53]:
strategies.WalkForwardUnanchored

strategies.WalkForwardUnanchored

In [56]:
bt_unanchored = Backtest(df, strategies.WalkForwardUnanchored, cash=100000, commission=.002, exclusive_orders=True)

stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

Backtest.optimize:   0%|          | 0/500 [00:00<?, ?it/s]

C:\Users\Dell\anaconda3\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [4, -3] before, using random point [4, -6]
  warnings.warn(


,limit_buy,limit_sell,Return [%]
0,0,-6,-100.0
1,0,-3,-100.0
22,5,-5,-100.0
21,4,-1,-100.0
20,4,-2,-100.0
19,4,-3,-100.0
18,4,-5,-100.0
17,4,-6,-100.0
16,3,-2,-100.0
15,3,-3,-100.0
